## update user contributions


In [ ]:
## should do this on a monthly basis because 1) this is will quickly use up BULK API's allotted quantity
## and 2) db2 only updates some usage numbers on a yearly or monthly basis

## currently this script takes over 25 minutes to upload to SF

In [87]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"shit9927289sSYTkLiGvyK2UrazHFmjFUGU",    
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [88]:
import sys
#sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Obtain information from DB2 

In [89]:
#pre-filtering queries using datetime
date_temp = datetime.datetime.today()
year = date_temp.year
month = date_temp.month-1
period = 14
if month < 10:
    date_str0 = str(year)+'-0'+str(month)+'-00'
    date_str1 = str(year)+'-0'+str(month)+'-01'
else:
    date_str0 = str(year)+'-'+str(month)+'-00'
    date_str1 = str(year)+'-'+str(month)+'-01'

In [90]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))

# getting the total number of contributions
sql_query = "select * from jos_real_contributors;"

num_contri_df = pd.read_sql_query(sql_query, engine)

# getting the users that used the contributions
## need to be selective here - only choose the previous month in the database, and pick period = 14
sql_query = "select * from jos_author_stats WHERE datetime between '2020-07-00' and '2020-07-01' and period = 14;"

result_users_df = pd.read_sql_query(sql_query, engine)


In [91]:
display(num_contri_df.head(2))
display(result_users_df.head(2))
print(result_users_df.shape)

,uidNumber,resource_count,wiki_count,total_count
0,-272,1,0,1
1,-271,0,0,0


,id,authorid,tool_users,andmore_users,total_users,datetime,period
0,1130819,4843,3118.0,869,3967,2020-07-00 00:00:00,14
1,1130822,28115,1840.0,0,1840,2020-07-00 00:00:00,14


(3121, 7)


In [92]:
## get rid of all negative uidNumbers
uids = num_contri_df['uidNumber']
remove = []
keep =[]
for index,val in enumerate(uids):
    if val <= 0:
        remove.append(index)
    else:
        keep.append(index)
contri_df = num_contri_df.iloc[keep,:]


## get rid of all negative authorid
ids = result_users_df['id']
remove = []
keep = []
for index,val in enumerate(ids):
    if val <= 0:
        remove.append(index)
    else:
        keep.append(index)
user_qty_df = result_users_df.iloc[keep,:]


In [93]:
display(contri_df.head(2))
display(user_qty_df.tail(2))

,uidNumber,resource_count,wiki_count,total_count
181,998,0,0,0
182,1000,0,0,0


,id,authorid,tool_users,andmore_users,total_users,datetime,period
3119,1140176,288999,3.0,0,3,2020-07-00 00:00:00,14
3120,1140179,288457,3.0,0,3,2020-07-00 00:00:00,14


In [94]:
print(contri_df.shape)
print(user_qty_df.shape)

(76401, 4)
(3121, 7)


In [95]:
# combine the two dataframes
ids = contri_df['uidNumber']
ovr_df = contri_df.set_index('uidNumber').join(user_qty_df.set_index('authorid'))

In [96]:
ovr_df = ovr_df.reset_index()
display(ovr_df.head(2))
display(ovr_df.tail(2))

,uidNumber,resource_count,wiki_count,total_count,id,tool_users,andmore_users,total_users,datetime,period
0,998,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


,uidNumber,resource_count,wiki_count,total_count,id,tool_users,andmore_users,total_users,datetime,period
76399,81151,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
76400,81153,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


## Get the current nanohub user IDs and their tool counts from SF
then begin the dataframe merge and post-processing

In [97]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [98]:
# get nanohub ID for contacts to being matching and processing
sf_username_df = db_s.query_data('SELECT nanoHUB_user_ID__c,Total_tool_contributions__c FROM Contact')

# display
display(sf_username_df.head(2))
print(sf_username_df.shape)

[Success] Bulk job creation successful. Job ID = 7505w00000Opm5GAAR
{"id":"7505w00000Opm5GAAR","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-08-19T23:02:14.000+0000","systemModstamp":"2020-08-19T23:02:14.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000Opm5GAAR","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-08-19T23:02:14.000+0000","systemModstamp":"2020-08-19T23:02:21.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000Opm5GAAR","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-08-19T23:02:14

,Total_tool_contributions__c,nanoHUB_user_ID__c
0,0.0,998.0
1,0.0,1683.0


(217015, 2)


In [99]:
# join the two dataframes
nover_df = sf_username_df.set_index('nanoHUB_user_ID__c').join(ovr_df.set_index('uidNumber'))

In [100]:
display(nover_df.head(2))

,Total_tool_contributions__c,resource_count,wiki_count,total_count,id,tool_users,andmore_users,total_users,datetime,period
998.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1683.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
## calculate the non tool contributions
# And_More_Contributions__c - total_count - tool_contributions
nover_df['And_More_Contributions__c'] = nover_df['total_count']-nover_df['Total_tool_contributions__c']

In [102]:
# recover nanohub ids 
nover_df = nover_df.reset_index()
display(nover_df.head(2))

,index,Total_tool_contributions__c,resource_count,wiki_count,total_count,id,tool_users,andmore_users,total_users,datetime,period,And_More_Contributions__c
0,998.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1683.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [112]:
## remove NAN rows for index
nover_df = nover_df.dropna(subset=['index'])
nover_df.shape

(216995, 12)

In [113]:
## then make the indexes integers
import numpy as np
nover_df['index'] = nover_df['index'].apply(lambda x: int(x))
display(nover_df.head(2))

,index,Total_tool_contributions__c,resource_count,wiki_count,total_count,id,tool_users,andmore_users,total_users,datetime,period,And_More_Contributions__c
0,998,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1683,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


## Fitting data to SF format

In [118]:
## creating and populating the sf dataframe
df_sf = pd.DataFrame()
temp_df = nover_df.fillna(0)

# populating df_sf and make integers
df_sf['nanoHUB_user_ID__c'] = temp_df['index'].apply(lambda x: int(x))
df_sf['And_More_Contributions__c'] = temp_df['And_More_Contributions__c'].apply(lambda x: int(x))
df_sf['contributed_tool_usage__c'] = temp_df['tool_users'].apply(lambda x: int(x))
df_sf['contributed_andmore_usage__c'] = temp_df['andmore_users'].apply(lambda x: int(x))

display(df_sf.head(2))

,nanoHUB_user_ID__c,And_More_Contributions__c,contributed_tool_usage__c,contributed_andmore_usage__c
0,998,0,0,0
1,1683,0,0,0


## send data to SF

In [119]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = 'Contact'
db_s.external_id = 'nanoHUB_user_ID__c'

Obtained Salesforce access token ...... True


In [120]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000OpmVCAAZ
hello
[Success] CSV upload successful. Job ID = 7505w00000OpmVCAAZ
[Success] Closing job successful. Job ID = 7505w00000OpmVCAAZ


In [129]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 21278941,
 'apiActiveProcessingTime': 1993465,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-08-19T23:12:28.000+0000',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'id': '7505w00000OpmVCAAZ',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 216995,
 'object': 'Contact',
 'operation': 'upsert',
 'retries': 0,
 'state': 'JobComplete',
 'systemModstamp': '2020-08-19T23:21:17.000+0000',
 'totalProcessingTime': 2188051}


In [130]:
pprint(db_s.check_bulk_failed_results())

'"sf__Id","sf__Error",And_More_Contributions__c,contributed_andmore_usage__c,contributed_tool_usage__c,nanoHUB_user_ID__c\n'
